In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Something about Bandit Problems
**Definitions**
- Bandit Problems
- Regret and other metrics
- Adversarial Bandit Problems

**Methods**
- Explore Only
- Exploit Only
- Epsilon-Greedy
- Upper Confidence Bound Algorithm

**TODO**
- Make charts of everything
- Tune parameters of mu and sigma to see how Regret converges as a function of time

**Goal:** Maximize value extracted within limited time and no given information

In [ ]:
# Review Latex and check out formulas for Regret and all that. It's probability stuff, so that's something else to learn

In [ ]:
class Restaurant:
    def __init__(self, mu, dev):
        self.mu = mu
        self.dev = dev
    def sample(self):
        return np.random.normal(self.mu, self.dev)

In [ ]:
# Graph a few here to show what you mean

In [ ]:
def explore_only(candidates, num_days):
    scores = []
    for _ in range(num_days):
        scores.append(choice(candidates).sample())
    return sum(scores)

In [ ]:
def exploit_only(candidates, num_days):
    scores = [c.sample() for c in candidates]
    chosen = candidates[np.argmax(scores)]
    for _ in range(num_days - len(candidates)):
        scores.append(chosen.sample())
    return sum(scores)

In [ ]:
def epsilon_greedy(candidates, num_days, epsilon=0.05):
    scores = []
    history = {idx: [c.sample()] for idx,c in enumerate(candidates)}
    for _ in range(num_days - len(candidates)):
        p = np.random.random()
        #explore
        if p < epsilon:
            chosen = choice(candidates)
        #exploit
        else:
            chosen = candidates[sorted(history.items(), key=lambda pair: np.mean(pair[1]))[-1][0]]
        score = chosen.sample()
        scores.append(score)
        history[candidates.index(chosen)].append(score)
    return sum(scores)

### Remember to cross-reference with other sources to check that you have the right formulas. Might be wrong

- Note: According to Auer (https://link.springer.com/article/10.1023/A:1013689704352), UCB1 experiments are initialized by having each machine played at least once (avoid divide by zero error). Time on top is separate from time on bottom (one is overall timestep, the other is time played)

In [ ]:
def ucb1(candidates, num_days):
    scores = []
    history = {idx: [c.sample()] for idx,c in enumerate(candidates)}
    for t in range(len(candidates), num_days):
        mu_plus_ucb = [np.mean(history[idx]) + np.sqrt(2*np.log(t) / len(history[idx])) for idx in range(len(candidates))]
        chosen = candidates[np.argmax(mu_plus_ucb)]
        
        score = chosen.sample()
        scores.append(score)
        history[candidates.index(chosen)].append(score)
    return sum(scores)

In [ ]:
dev_factor = 0.5
num_restaurants = 3

mu_vals = [3*i for i in range(1,num_restaurants+1)]
dev_vals = [mu*dev_factor for mu in mu_vals]
mu_dev_pairs = zip(mu_vals, dev_vals)

candidates = [Restaurant(mu,dev) for mu,dev in mu_dev_pairs]

num_days = 300

optimal_average = max(mu_vals)*num_days

In [ ]:
explore_only_vals = []
for _ in range(1000):
    val = explore_only(candidates, num_days)
    explore_only_vals.append(val)
print('Explore Only Mean Regret: %s'%((optimal_average - np.mean(explore_only_vals)) / optimal_average))

In [ ]:
exploit_only_vals = []
for _ in range(1000):
    val = exploit_only(candidates, num_days)
    exploit_only_vals.append(val)
print('Exploit Only Mean Regret: %s'%((optimal_average - np.mean(exploit_only_vals)) / optimal_average))

In [ ]:
epsilon_greedy_vals = []
for _ in range(1000):
    val = epsilon_greedy(candidates, num_days, 0.1)
    epsilon_greedy_vals.append(val)
print('Epsilon Greedy Mean Regret (10%%): %s'%((optimal_average - np.mean(epsilon_greedy_vals)) / optimal_average))